In [1]:
# Libraries
import csv
import pandas as pd
import matplotlib.pyplot as plt

# Read in data
#Cloud_Sections
cs = pd.read_csv('CLOUD_SECTIONS_Cleaned.csv', delimiter = ',', encoding = 'latin1')
#Designed_Pipe_Center
desP = pd.read_csv('DESIGNED_Pipe_Centers_Cleaned.csv', delimiter = ',', encoding = 'latin1')
#Planes_Local_Global_Cleaned
p_info = pd.read_csv('Planes_Local_Global_Cleaned.csv', delimiter = ',', encoding = 'latin1')

In [2]:
SectionedList = []
XsecList = []
YsecList = []
for s in range(cs.Section.nunique()): #creates a List of dataframes where each dataframe is a section
    SectionedList.append(cs[cs["Section"] == "Section-{" + str(s) + "}"])

for s in SectionedList: #for Section in SectionList, s is a dataframe for a section
    x = s.X
    y = s.Y
    XsecList.append(list(x))
    YsecList.append(list(y))
    #print(s)
    #plt.hist(s.X, bins=8, color = "orange")
    #print("Histogram of x values")
    #plt.show()
    #print(s.X)
   
    #plt.scatter(x, y)
    #plt.show()


In [3]:
#create list showing the pipe centers
CenterList = []
for n in range(desP.Section.nunique()): #creates a List of dataframes where each dataframe is a section
    CenterList.append(desP[desP["Section"] == "Section-{" + str(n) + "}"])

In [ ]:
TuplePairsBySection = []
for i, j in zip(XsecList, YsecList):
    TuplePairsBySection.append(list(zip(i, j)))
count = 0
for C in TuplePairsBySection:
    plt.title("Section " + str(count))
    plt.xlim(0, max(x[0] for x in C) * 1.1)
    plt.ylim(0, max(y[1] for y in C) * 1.1)
    plt.scatter(*zip(*C))
    plt.scatter(CenterList[count].X, CenterList[count].Y, color = "red")
    plt.show()
    count += 1

In [ ]:
#OLD VERSION
count = 0
for i, j in zip(XsecList, YsecList):   
    plt.title("Section " + str(count))
    plt.xlim(0, max(i) + max(i) * 0.1)
    plt.ylim(0, max(j) + max(j) * 0.1)
    plt.scatter(i, j)
    plt.scatter(CenterList[count].X, CenterList[count].Y, color = "red")
    for n in range(len(CenterList[count])):
        #print(CenterList[count].iloc[n,])
        plt.Circle((CenterList[count].iloc[n,1], CenterList[count].iloc[n,2]), CenterList[count].iloc[n,4])
        plt.show()
    plt.show()
    count += 1
print("done")